In [1]:
import numpy as np
import ufl
from mpi4py import MPI
import dolfinx.fem.petsc
import dolfinx

In [2]:
omega = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 30, 30)
V = dolfinx.fem.FunctionSpace(omega, ("Lagrange", 1))

In [3]:
dt = dolfinx.fem.Constant(omega, 0.01)
dt.value = 0.005

In [4]:
def k_func(t):
    return 0.1 if t < 0.5 else 0.05


k = dolfinx.fem.Constant(omega, 0.1)
t = dolfinx.fem.Constant(omega, 0.)
while t.value < 1:
    # Update t
    t.value += dt.value
    # Update k
    k.value = k_func(t.value)
# We reset t
t.value = 0

In [5]:
x, y = ufl.SpatialCoordinate(omega)

# Next, we create each component of the conditional statement
condition = ufl.lt(x, 0.5)
true_statement = 0.4 * y
false_statement = 0.5 * t
f = ufl.conditional(condition, true_statement, false_statement)

In [6]:
u = ufl.TrialFunction(V)
u_n = dolfinx.fem.Function(V)
dudt = (u - u_n) / dt

v = ufl.TestFunction(V)
dx = ufl.Measure("dx", domain=omega)
F = dudt * v * dx + k * ufl.inner(ufl.grad(u), ufl.grad(v)) * dx - f * v * dx
a, L = ufl.system(F)

# We generate and compile the C code for these expressions using `dolfinx.fem.form`


# + tags=["hide-output"]
a_compiled = dolfinx.fem.form(a)
L_compiled = dolfinx.fem.form(L)

In [7]:
def u_init(x, sigma=0.1, mu=0.3):
    """
    The input function x is a (3, number_of_points) numpy array, which is then
    evaluated with the vectorized numpy functions for efficiency
    """
    return 1./(2 * np.pi * sigma)*np.exp(-0.5*((x[0]-mu)/sigma)**2)*np.exp(-0.5*((x[1]-mu)/sigma)**2)


u_n.interpolate(u_init)

In [8]:
uh = dolfinx.fem.Function(V)
petsc_options = {"ksp_type": "preonly",
                 "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"}
problem = dolfinx.fem.petsc.LinearProblem(
    a_compiled, L_compiled, u=uh, bcs=[], petsc_options=petsc_options)

# For each temporal step, we update the time variable and call the `solve` command that re-assemble the system

T = 1
while t.value < T:
    t.value += dt.value
    k.value = k_func(t.value)
    problem.solve()
    # Update previous solution
    u_n.x.array[:] = uh.x.array

In [9]:
uh.x.array

array([0.31771159, 0.31735563, 0.31723309, 0.31763302, 0.31891261,
       0.31701072, 0.31884741, 0.31742222, 0.32088341, 0.31666548,
       0.32082477, 0.31864616, 0.3170808 , 0.32352327, 0.31618861,
       0.32346999, 0.32063097, 0.31831051, 0.31660551, 0.32669303,
       0.31557499, 0.32664563, 0.32328365, 0.32030181, 0.31783869,
       0.31599256, 0.33020895, 0.31482091, 0.33016887, 0.32646869,
       0.32296273, 0.31983522, 0.31722765, 0.31523854, 0.33383735,
       0.31392385, 0.33380678, 0.33000512, 0.32615969, 0.32250429,
       0.31922796, 0.31647415, 0.31434099, 0.3372887 , 0.31288286,
       0.3372707 , 0.33366205, 0.32971423, 0.3257146 , 0.32190438,
       0.31847656, 0.31557565, 0.31329889, 0.34021095, 0.31169895,
       0.34020957, 0.33715323, 0.33339859, 0.32929084, 0.32512832,
       0.32115877, 0.31757808, 0.31453095, 0.31211323, 0.34218201,
       0.31037567, 0.34220237, 0.34013043, 0.33693032, 0.33300937,
       0.32872834, 0.32439549, 0.32026381, 0.31653092, 0.31334